In [1]:
from graphviz import Digraph
def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._downstream:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ data %.4f | grad %.4f }" % (n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    return dot

In [7]:
from nanotensor.module.engine.tensor import Tensor
from nanotensor.module.model import layers, loss
import matplotlib.pyplot as plt
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [13]:
iris = datasets.load_iris()
X = iris.data  
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)

In [11]:
#There are four features and 3 classes, the network will be setup to solve a multiclass problem
model = layers.Sequential([layers.Layer((4,20)),
                layers.Layer((20,20) ,activation='relu'),
                layers.Layer((20,10) ,activation='relu'),
                layers.Layer((10,3) ,activation='sigmoid')])
model.summary()

# for i in range(50):
#     ypred = [model(x) for x in X]
#     loss = sum([(yhat-y)**2 for yhat,y in zip(ypred,y)]) / len(ypred)
#     if i in range(0,500,25):
#         print(f'Iteration:{i} ---> {loss.data:}')
#     lr = 0.01
#     loss.zero_grad()
#     loss.backward()
#     for p in model.parameters():
#         p.data += -(lr * p.grad)

Layer(type)         Shape              Parameters
---------------------------------------------------
LinearLayer       Shape: (4, 20))      Params: 100
---------------------------------------------------
ReLULayer       Shape: (20, 20))      Params: 420
---------------------------------------------------
ReLULayer       Shape: (20, 10))      Params: 210
---------------------------------------------------
SigmoidLayer       Shape: (10, 3))      Params: 33
---------------------------------------------------
Trainable Parameters                          763


In [18]:
model([5,7,50,55,60,60])

[Tensor(data= (1.0), grad=(0.0)),
 Tensor(data= (1.0), grad=(0.0)),
 Tensor(data= (1.0), grad=(0.0))]